In [29]:
import numpy as np
import pandas as pd
import os.path
from LOB_utils import ScalerNew,StandardScaler,MinMaxScaler,readTable,readTable,rollingBeforeK,rollingAfterBeforeK,rollingAfterK,labelBasedOnRate

# 输入数据集的结构 root/year/month/day/stock_code.csv
# 输出处理过后数据集结构 root/stock/year/month/day.csv
# 读取某一天的数据并且返回dataframe 与这个数据集的结构相关


ImportError: cannot import name 'ScalerNew' from 'LOB_utils' (D:\新建文件夹\教材\创新实践2\process_data\LOB_utils.py)

In [ ]:
def readGP(tiaomu,month,i):
    data_path = ''
    day = ''
    if i<=9:
        day+='0'
    data_path = 'D:\\新建文件夹\\教材\\创新实践2\\2015{}\\2015{}{}\\{}{}{}-L1.CSV'.format(month,month,day+str(i),tiaomu,month,day+str(i))
    if not os.path.isfile(data_path):
        print(data_path)
        return
    data = pd.read_csv(data_path)
    return data

In [ ]:
# 数据集中所需要的股票的代码
tiaomu = ['000002.SZ1-Tick-2015','000063.SZ1-Tick-2015','000069.SZ1-Tick-2015','000100.SZ1-Tick-2015','000157.SZ1-Tick-2015','000166.SZ1-Tick-2015','000301.SZ1-Tick-2015','000333.SZ1-Tick-2015','000338.SZ1-Tick-2015','000408.SZ1-Tick-2015']


#计算各标签的比例
def calRate01(rate10,yuzhi):
    rate_ = rate10[(rate10 >= -yuzhi) & (rate10 <= yuzhi)].shape[0] / rate10.shape[0]
    rateda = rate10[(rate10 <= -yuzhi)].shape[0] / rate10.shape[0]
    ratexiao = rate10[(rate10 >= yuzhi)].shape[0] / rate10.shape[0]
    print(rate_,ratexiao,rateda)


In [ ]:
def processD(tiaomu,mon,day):
    #month day是某一天的数据
    #tiaomu是我所需要的股票名称
    #data是某只股票某一天的数据
    data = readGP(tiaomu,mon,day)


    if data is None:
        return
    col_name = ['AskPrice1','AskVolume1','AskPrice2','AskVolume2','AskPrice3','AskVolume3','AskPrice4','AskVolume4','AskPrice5','AskVolume5','BidPrice1','BidVolume1','BidPrice2','BidVolume2','BidPrice3','BidVolume3','BidPrice4','BidVolume4','BidPrice5','BidVolume5','Price']
    # 留下我所需要的列
    data = data[col_name]

    # 留下我所需要的行
    # 去除最开始那一段竞拍时间点,以及涨停跌停的数据
    data = data[data['Price']!=0]
    data = data[(data['AskPrice1'] != 0)&(data['BidPrice1'] != 0)&(data['AskPrice2'] != 0)&(data['BidPrice2'] != 0)]
    realPrice = data['Price'].copy()
    if data.shape[0]==0:
        return
    datatest=data.copy()
    price = ['AskPrice1','AskPrice2','AskPrice3','AskPrice4','AskPrice5','BidPrice1','BidPrice2','BidPrice3','BidPrice4','BidPrice5']
    volome = ['AskVolume1','AskVolume2','AskVolume3','AskVolume4','AskVolume5','BidVolume1','BidVolume2','BidVolume3','BidVolume4','BidVolume5']

    # 使用minmax进行标准化
    scaler = MinMaxScaler()
    p_normal = scaler.fit_transform(datatest[price])
    if scaler.min_val<=0:
        return
    v_normal = scaler.fit_transform(datatest[volome])
    data = pd.concat([p_normal,v_normal],axis=1)

    # 计算中间价
    data['curPrice'] = (data['AskPrice1']+data['BidPrice1'])/2

    # 将每一个值与它之前k个数的平均值取差，然后除以自己算出增长率。  下面的1 2 3 5 10 50 100 500就是k
    # 现在是现在与过去比，另有两个方法，即未来与现在比，还有未来与过去比。
    rate1 = rollingBeforeK(data['curPrice'],1)
    rate2 = rollingBeforeK(data['curPrice'],2)
    rate5 = rollingBeforeK(data['curPrice'],5)
    rate10 = rollingBeforeK(data['curPrice'],10)


    if rate1.shape[0]==0:
        print(tiaomu,day,mon)
        return


    data['rate1']=rate1
    data['rate2']=rate2
    data['rate5']=rate5
    data['rate10']=rate10


    print("各标签比例 {} {} {}".format(tiaomu,mon,day)) #打印当前股票在这个月的各标签比例
    #计算、打印标签比例
    active1 = 0.0001
    active2 = 0.005
    active5 = 0.01
    active10 = 0.012

    calRate01(rate1,active1)
    calRate01(rate2,active2)
    calRate01(rate5,active5)
    calRate01(rate10,active10)

    # condition内保存具体条件，可以做n分类
    conditions1 = [
        (data['rate1'] >= -active1) & (data['rate1'] <= active1),
        data['rate1'] > active1,
        data['rate1'] < -active1
    ]
    conditions2 = [
        (data['rate2'] >= -active2) & (data['rate2'] <= active2),
        data['rate2'] > active2,
        data['rate2'] < -active2
    ]
    conditions5 = [
        (data['rate5'] >= -active5) & (data['rate5'] <= active5),
        data['rate5'] > active5,
        data['rate5'] < -active5
    ]
    conditions10 = [
        (data['rate10'] >= -active10) & (data['rate10'] <= active10),
        data['rate10'] > active10,
        data['rate10'] < -active10
    ]
    # choice里面的标签数量与condition一致即可
    choices = [0,1,-1]


    #根据上面计算得到的阈值（使得三种标签比较平衡），将不同k的标签打上。 可以实现多标签，只需改动上面两个变量长度
    data['label_for_1'] = labelBasedOnRate(conditions1,choices)
    data['label_for_2'] = labelBasedOnRate(conditions2,choices)
    data['label_for_5'] = labelBasedOnRate(conditions5,choices)
    data['label_for_10'] = labelBasedOnRate(conditions10,choices)



#输出结果
    if not os.path.exists('D:\\新建文件夹\\教材\\创新实践2\\train1\\{}\\2015\\{}'.format(tiaomu,mon)):
        os.makedirs('D:\\新建文件夹\\教材\\创新实践2\\train1\\{}\\2015\\{}'.format(tiaomu,mon))
    data.to_csv('D:\\新建文件夹\\教材\\创新实践2\\train1\\{}\\2015\\{}\\{}.csv'.format(tiaomu,mon,day))




In [ ]:
# processD(tiaomu[0],'05',4)

In [30]:
for t in tiaomu:
    for mon in ['05','06','07']:
        for day in range(1,31):
            processD(t,mon,day)


D:\新建文件夹\教材\创新实践2\201505\20150501\000002.SZ1-Tick-20150501-L1.CSV
D:\新建文件夹\教材\创新实践2\201505\20150502\000002.SZ1-Tick-20150502-L1.CSV
D:\新建文件夹\教材\创新实践2\201505\20150503\000002.SZ1-Tick-20150503-L1.CSV
14.83 14.37
3554305 20
各标签比例 000002.SZ1-Tick-2015 05 4
0.7319283456269757 0.13466807165437303 0.13319283456269757
0.589041095890411 0.20611169652265543 0.20463645943097997
0.41791359325605903 0.2874604847207587 0.29441517386722865
0.3108535300316122 0.3426765015806112 0.346259220231823
14.65 13.59
3198700 20
各标签比例 000002.SZ1-Tick-2015 05 5
0.7085353003161222 0.1399367755532139 0.15131717597471023
0.5747102212855637 0.2 0.2250790305584826
0.5738672286617492 0.19114857744994732 0.23477344573234984
0.5262381454162276 0.20463645943097997 0.26891464699683876
14.09 13.37
1957100 20
各标签比例 000002.SZ1-Tick-2015 05 6
0.7076923076923077 0.14668071654373024 0.14541622760800843
0.5909378292939936 0.20126448893572182 0.20758693361433087
0.5888303477344573 0.19599578503688092 0.21496311907270813
0.55405690